In [7]:
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime
import re

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

con = None
try:
    
    logging.basicConfig(filename = 'CommonConfigLog.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    
    def open_connection():
        global server, database, driver, connection, con
        with open(r'C:\Users\gmoye001\configtest\config.json', 'r') as fh:
            config = json.load(fh)
        server = config['server']
        database = config['database']
        driver = config['driver']
        connection = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes'
        con = pyodbc.connect(connection)
        return con
    
    def connection_test():
        cur = con.cursor()
        cur.execute("SELECT @@version")
        row = cur.fetchone()
        print("Connection established to: ",row[0])
        cur.close()
        con.commit()
        logging.info(f"\nConnection established to: {row[0]}\n")
        return print(f"Connection established to: {row[0]}")
        
    def close_connection():
        con.close()
        return
    
    def checktables(con, tbl):
        if stage == False:
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_name = '%s'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n{tbl} does not exist in the database and will need to be created\n'
                print(output, end = "\r")
                logging.info(output, end = "\r")
                cur.close()
                return False
            else:
                print(output, end = "\r")
                cur.close()
            return True
        else:
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_name = '%s_stage'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n{tbl}_stage does not exist in the database and will need to be created\n'
                print(output, end = "\r")
                logging.info(output, end = "\r")
                cur.close()
                return False
            else:
                print(output, end = "\r")
                cur.close()
            return True

    
    def checkbackups(con, tbl):
        if stage == False:
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_name = '%s_backup'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n{tbl}_backup does not exist in the database and will need to be created once parent table has data\n'
                print(output, end = "\r")
                logging.info(output)
                cur.close()
                return False
            else:
                print(f'\nCurrent backup for {tbl} exists.', end = "\r")
                logging.info(f'\nCurrent backup for {tbl} exists.\n')
                cur.close()
                return True
        else:
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_name = '%s_stage-backup'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n{tbl}_stage-backup does not exist in the database and will need to be created once parent table has data\n'
                print(output, end = "\r")
                logging.info(output)
                cur.close()
                return False
            else:
                print(f'\nCurrent backup for {tbl}_stage exists.', end = "\r")
                logging.info(f'\nCurrent backup for {tbl}_stage exists.\n')
                cur.close()
                return True
    
    def tablecontent(con, tbl):
        if stage == False:
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [%s]" % tbl
            cur.execute(query)
            output = cur.fetchone()
            #print(output)
            if output[0] == 0:
                print(f'{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return False
            else:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return True
        else:
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [%s_stage]" % tbl
            cur.execute(query)
            output = cur.fetchone()
            #print(output)
            if output[0] == 0:
                print(f'{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return False
            else:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return True
        
    def backup_data(con, tbl):
        if stage == False:
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "INSERT INTO [%s_backup] SELECT * FROM [%s]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                duration = bend_time - bstart_time
                logging.info(f'\nBackup Duration: {duration}')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return
        else:
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "INSERT INTO [%s_stage-backup] SELECT * FROM [%s_stage]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                duration = bend_time - bstart_time
                logging.info(f'\nBackup Duration: {duration}')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return
        
    def create_backuptable(con, tbl):
        if stage == False:
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "SELECT * INTO [%s_backup] FROM [%s]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                duration = bend_time - bstart_time
                logging.info(f'\nBackup Duration: {duration}')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
                return False
            return True
        else:
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "SELECT * INTO [%s_stage-backup] FROM [%s_stage]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                duration = bend_time - bstart_time
                logging.info(f'\nBackup Duration: {duration}')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
                return False
            return True
        
    def backupcheck(con, tbl):
        global rowcount
        if stage == False:
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [%s] UNION ALL SELECT COUNT(*) FROM [%s_backup]" % (tbl, tbl)
            cur.execute(query)
            output = cur.fetchone()
            rowcount = []
            while output is not None:
                rowcount.append(output[0])
                output = cur.fetchone()
            print('\n# of records in each table: ', rowcount[0], recnum, end = "\r")
            logging.info(f'\n# of records in each table: {rowcount[0]}, {recnum}')
            cur.close()
            if recnum == rowcount[0]:
                truncate_backup(con, tbl)
                backup_data(con, tbl)
            elif recnum < rowcount[0]:
                truncate_backup(con, tbl)
                backup_data(con, tbl)
            elif rowcount[0] == 0:
                pass
            return rowcount
        else:
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [%s_stage] UNION ALL SELECT COUNT(*) FROM [%s_stage-backup]" % (tbl, tbl)
            cur.execute(query)
            output = cur.fetchone()
            rowcount = []
            while output is not None:
                rowcount.append(output[0])
                output = cur.fetchone()
            print('\n# of records in each table: ', rowcount[0], recnum, end = "\r")
            logging.info(f'\n# of records in each table: {rowcount[0]}, {recnum}')
            cur.close()
            if recnum == rowcount[0]:
                truncate_backup(con, tbl)
                backup_data(con, tbl)
            elif recnum < rowcount[0]:
                truncate_backup(con, tbl)
                backup_data(con, tbl)
            elif rowcount[0] == 0:
                pass
            return rowcount
    
    def truncate_table(con, tbl):
        if stage == False:
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [%s]" % tbl
                cur.execute(query)
                print(f'\n{tbl} has been succesfully truncated to import new data.')
                logging.info(f'\n{tbl} has been succesfully truncated to import new data.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
        else:
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [%s_stage]" % tbl
                cur.execute(query)
                print(f'\n{tbl}_stage has been succesfully truncated to import new data.')
                logging.info(f'\n{tbl}_stage has been succesfully truncated to import new data.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
            
    def truncate_backup(con, tbl):
        if stage == False:
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [%s_backup]" % tbl
                cur.execute(query)
                print(f'\n{tbl}_backup has been succesfully truncated to create the next backup.')
                logging.info(f'\n{tbl}_backup has been succesfully truncated to create the next backup.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
        else:
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [%s_stage-backup]" % tbl
                cur.execute(query)
                print(f'\n{tbl}_stage-backup has been succesfully truncated to create the next backup.')
                logging.info(f'\n{tbl}_stage-backup has been succesfully truncated to create the next backup.\n')
                cur.close()
                con.commit()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err

    def importdata(con, tbl):
        if stage == False:
            cur = con.cursor()
            engine = sa.create_engine(f"mssql+pyodbc://{server}/{database}?driver={driver}?Trusted_Connection=yes", fast_executemany = True)
            pd.io.sql._is_sqlalchemy_connectable(engine)
            df.to_sql(f'{tbl}', engine, index = False, if_exists = 'append', schema = 'dbo')
            return
        else:
            cur = con.cursor()
            engine = sa.create_engine(f"mssql+pyodbc://{server}/{database}?driver={driver}?Trusted_Connection=yes", fast_executemany = True)
            pd.io.sql._is_sqlalchemy_connectable(engine)
            df.to_sql(f'{tbl}_stage', engine, index = False, if_exists = 'append', schema = 'dbo')
            
    def getData(CV):
        # convert to config file/table
        if stage == False:
            url = f'https://api.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}'
            with open(r'C:\Users\gmoye001\configtest\apiconnect.json') as f:
                headers = json.load(f)
        else:
            url = f'https://api-staging.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}'
            with open(r'C:\Users\gmoye001\configtest\apiconnectb.json') as f:
                headers = json.load(f)
    
        
        r = requests.get(url, headers=headers)
        rjson = r.json()
        keylist = ('URI','Categories','RelatedTerms')
        
        for key in keylist:
            rjson = [{k: v for k, v in d.items() if k != key} for d in rjson]
        
        global df
        df = pd.DataFrame(rjson)
        datelist = ('CreatedDate','ModifiedDate','EffectiveDate', 'RelModifiedDate','ExpiryDate','Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime')
        date_format = "%Y%m%d%H%M%S"
        
        for date in enumerate(datelist):
            if date[1] in df:
                df[date[1]] = df[date[1]].str.replace("\.[0-9]*Z", "").str.replace("Z", "")
                if date[1] != 'None':
                    df[date[1]] = pd.to_datetime(df[date[1]], format=date_format, errors = 'coerce')
                else:
                    continue
            else:
                continue
                
        global recnum
        recnum = len(df.index)
        
        #display(df)
        #df.to_excel('output1.xlsx')
        
        if tablecontent(con, f'{CV}') == False:
            importdata(con, f'{CV}')
            print(f'\n{CV} has been updated with {recnum} records')
            logging.info(f'\n{CV} has been updated with {recnum} records\n')
        else:
            truncate_table(con, f'{CV}')
            importdata(con, f'{CV}')
            print(f'\n{CV} has been updated with {recnum} records')
            logging.info(f'\n{CV} has been updated with {recnum} records\n')
            
        #colnames = list(df)
        
        #display(colnames) 
         #[['Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime']])
        
        #--fetch column names for table creation and datatypes
        #for colname, dt in itertools.product([df.columns],[df.dtypes]):
            #print(dt)
        #print('\n')
        
        return df
    
    
    
    with open(r'C:\Users\gmoye001\configtest\CVsStage.csv', 'r') as cv_config:
        CVsStage = cv_config.read().split(',')
    with open(r'C:\Users\gmoye001\configtest\CVsProd.csv', 'r') as cv_config:
        CVsProd = cv_config.read().split(',')

    #Used for testing a group of CV's
    CV = ['LEL-PwCLegalEntity-en','NS-PwCNetworkNode-en',
           'NS-PwCNetworkNode-en-Territory',
           'ORD-CostCenter']
    #Used for testing a single CV
    CVx = ['ORD-CostCenter']
    
    CVd = ['ORD-CostCenter']
    
    st = datetime.now()
    open_connection()
    connection_test()
    close_connection()
    
    for urls in enumerate(CVsStage):
        global stage
        stage = True
        open_connection()
        #getData(urls[1])
        #close_connection()
        if checktables(con, urls[1]) == False:
            print("Moving to next table.\n", end = "\r")
        else:
            if checkbackups(con, urls[1]) == False:
                create_backuptable(con, urls[1])
                createdbackup = True
            else:
                createdbackup = False
            try:
                gstart_time = datetime.now()
                getData(urls[1])
                if createdbackup == False:
                    backupcheck(con, urls[1])
                else:
                    pass
                close_connection()
                stage = False
                gend_time = datetime.now()
                print('\nDuration: {}'.format(gend_time - gstart_time))
                logging.info('\nDuration: {}'.format(gend_time - gstart_time))
                print("")
            except (Exception, pyodbc.DatabaseError) as e:
                print("")
                print(e)
                logging.exception('\n')
                logging.exception("message")
        for i in range(10, -1, -1):
                print(f"{i} seconds until next table is imported ", end = "\r")
                ti.sleep(1)

    for urls in enumerate(CVsProd):
        open_connection()
        stage = False
        #getData(urls[1])
        #close_connection()
        if checktables(con, urls[1]) == False:
            print("Moving to next table.\n", end = "\r")
        else:
            if checkbackups(con, urls[1]) == False:
                create_backuptable(con, urls[1])
                createdbackup = True
            else:
                createdbackup = False
            try:
                gstart_time = datetime.now()
                getData(urls[1])
                if createdbackup == False:
                    backupcheck(con, urls[1])
                else:
                    pass
                close_connection()
                gend_time = datetime.now()
                print('\nDuration: {}'.format(gend_time - gstart_time))
                logging.info('\nDuration: {}'.format(gend_time - gstart_time))
                print("")
            except (Exception, pyodbc.DatabaseError) as e:
                print("")
                print(e)
                logging.exception('\n')
                logging.exception("message")
        for i in range(10, -1, -1):
                print(f"{i} seconds until next table is imported ", end = "\r")
                ti.sleep(1)
            
    et = datetime.now()
    print('Total Execution Duration: {}'.format(et - st),'\n-Import Completed-')
    tt = et - st
    logging.info(f'\nTotal Execution Duration: {tt}\n')
    logging.info('\n-Import Completed-\n')
    
#exceptions 
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except r.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    logging.exception("message")
    
except r.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    logging.exception("message")
    
except r.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    logging.exception("message")
    
except r.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')

Connection established to:  Microsoft SQL Server 2016 (SP2) (KB4052908) - 13.0.5026.0 (X64) 
	Mar 18 2018 09:11:49 
	Copyright (c) Microsoft Corporation
	Enterprise Edition (64-bit) on Windows Server 2016 Datacenter 10.0 <X64> (Build 14393: ) (Hypervisor)

Connection established to: Microsoft SQL Server 2016 (SP2) (KB4052908) - 13.0.5026.0 (X64) 
	Mar 18 2018 09:11:49 
	Copyright (c) Microsoft Corporation
	Enterprise Edition (64-bit) on Windows Server 2016 Datacenter 10.0 <X64> (Build 14393: ) (Hypervisor)

[('CommonConfig', 'dbo', 'Onboarding_ORD-CostCentreLegalEntity-en-GlobalHierarchy_Denormalised_stage', 'BASE TABLE')]
Current backup for Onboarding_ORD-CostCentreLegalEntity-en-GlobalHierarchy_Denormalised_stage exists.
6605 Existing records
Onboarding_ORD-CostCentreLegalEntity-en-GlobalHierarchy_Denormalised_stage has been succesfully truncated to import new data.

Onboarding_ORD-CostCentreLegalEntity-en-GlobalHierarchy_Denormalised has been updated with 6605 records

# of records 